In [1]:
%pip install pandas numpy matplotlib seaborn pyarrow fastparquet
%pip uninstall pyarrow pandas -y
%pip cache purge
%pip install pandas==2.1.4 pyarrow==14.0.2


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Found existing installation: pyarrow 14.0.2
Uninstalling pyarrow-14.0.2:
  Successfully uninstalled pyarrow-14.0.2
Found existing installation: pandas 2.1.4
Uninstalling pandas-2.1.4:
  Successfully uninstalled pandas-2.1.4
Note: you may need to restart the kernel to use updated packages.
Files removed: 12
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB 1.3 MB/s eta 0:00:09
   ---- ----------------------------------- 1.1/10.6 MB 17.1 MB/s eta 0:00:01
   ------------------ --------------------- 5.0/10.6 MB 45.4 MB/s eta 0:00:01
   ----------------------- ---------------- 6.3/10.6 MB 44.5 MB/s eta 0:00:01
   ---------------------------------------  10.6/10.6 MB 73.1 MB/s eta 0:00:01
   ---------------------------------------- 10.6/10.6 MB 65.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.6 MB ? eta -:--:--
   ----- --------


[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import os
import json

In [ ]:
# Cargamos solo el año 2024 para entender la estructura:

year = 2024
base_path = f'../data/raw/datos/vct_{year}'

print(f"Cargando datos de {year} ...")
print("=" *60)

# Cargamos overview.csv ya que es el que muestra un resumen de todo
df_overview = pd.read_csv(f'{base_path}/matches/overview.csv')
print(f' Overview cargado:{df_overview.shape[0]:,} filas x {df_overview.shape[1]} columnas')

display(df_overview.head(3))

Cargando datos de 2024 ...
 Overview cargado:46,152 filas x 21 columnas


,Tournament,Stage,Match Type,Match Name,Map,Player,Team,Agents,Rating,Average Combat Score,...,Deaths,Assists,Kills - Deaths (KD),"Kill, Assist, Trade, Survive %",Average Damage Per Round,Headshot %,First Kills,First Deaths,Kills - Deaths (FKD),Side
0,Valorant Champions 2024,Group Stage,Opening (B),Gen.G vs Sentinels,Haven,t3xture,Gen.G,jett,1.60,305.0,...,12.0,4.0,12.0,81%,188.0,31%,3.0,4.0,-1.0,both
1,Valorant Champions 2024,Group Stage,Opening (B),Gen.G vs Sentinels,Haven,t3xture,Gen.G,jett,1.71,364.0,...,5.0,2.0,7.0,89%,216.0,37%,2.0,2.0,0.0,attack
2,Valorant Champions 2024,Group Stage,Opening (B),Gen.G vs Sentinels,Haven,t3xture,Gen.G,jett,1.52,261.0,...,7.0,2.0,5.0,75%,168.0,24%,1.0,2.0,-1.0,defend


In [ ]:
print(" COLUMNAS DISPONIBLES EN OVERVIEW.CSV:")
print("=" * 60)

# Listar todas las columnas
for i, col in enumerate(df_overview.columns, 1):
    print(f"{i:2}. {col}")

print(f"\nTotal: {len(df_overview.columns)} columnas")

 COLUMNAS DISPONIBLES EN OVERVIEW.CSV:
 1. Tournament
 2. Stage
 3. Match Type
 4. Match Name
 5. Map
 6. Player
 7. Team
 8. Agents
 9. Rating
10. Average Combat Score
11. Kills
12. Deaths
13. Assists
14. Kills - Deaths (KD)
15. Kill, Assist, Trade, Survive %
16. Average Damage Per Round
17. Headshot %
18. First Kills
19. First Deaths
20. Kills - Deaths (FKD)
21. Side

Total: 21 columnas


In [ ]:
print(" ANÁLISIS DE ESTRUCTURA")
print("=" * 60)

# ¿Cuántos partidos únicos hay?
unique_matches = df_overview['Match Name'].nunique()
print(f"\n Partidos únicos: {unique_matches:,}")

# ¿Cuántos jugadores únicos?
unique_players = df_overview['Player'].nunique()
print(f" Jugadores únicos: {unique_players:,}")

# ¿Cuántos equipos únicos?
unique_teams = df_overview['Team'].nunique()
print(f"  Equipos únicos: {unique_teams:,}")

# ¿Cuántas filas totales?
total_rows = len(df_overview)
print(f" Total de filas: {total_rows:,}")

# Veamos un partido específico
print("\n" + "=" * 60)
print(" EJEMPLO: Veamos el primer partido completo")
print("=" * 60)

first_match = df_overview['Match Name'].iloc[0]
first_match_data = df_overview[df_overview['Match Name'] == first_match]

print(f"\nPartido: {first_match}")
print(f"Filas para este partido: {len(first_match_data)}")
print(f"Jugadores: {first_match_data['Player'].nunique()}")
print(f"Equipos: {first_match_data['Team'].unique()}")

print("\nPrimeras 5 filas de este partido:")
display(first_match_data[['Player', 'Team', 'Map', 'Kills', 'Deaths']].head())

 ANÁLISIS DE ESTRUCTURA

 Partidos únicos: 329
 Jugadores únicos: 260
  Equipos únicos: 44
 Total de filas: 46,152

 EJEMPLO: Veamos el primer partido completo

Partido: Gen.G vs Sentinels
Filas para este partido: 480
Jugadores: 10
Equipos: ['Gen.G' 'Sentinels']

Primeras 5 filas de este partido:


,Player,Team,Map,Kills,Deaths
0,t3xture,Gen.G,Haven,24.0,12.0
1,t3xture,Gen.G,Haven,12.0,5.0
2,t3xture,Gen.G,Haven,12.0,7.0
3,Meteor,Gen.G,Haven,17.0,10.0
4,Meteor,Gen.G,Haven,8.0,4.0


In [ ]:
print(" Análisis de mapas")
print("=" *60)

# Vemos cuantos mapas únicos hay
unique_maps = df_overview['Map'].unique()
print(f"\n Mapas únicos en el juego:  {unique_maps}")
print(f"Mapas disponibles: {df_overview['Map'].unique()[:10]}")

# Vemos el primer partido 
first_match = df_overview['Match Name'].iloc[0]
first_match_data = df_overview[df_overview['Match Name'] == first_match]

maps_in_match = first_match_data['Map'].unique()
print(f"\n Partido: {first_match}")
print(f"Mapas jugados: {len(maps_in_match)}")
print(f"Cuales: {maps_in_match}")

# Vemos la columna side
print(f" Valores de Side: {df_overview['Side'].unique()}")

# Contamos las filas por mapa en el primer partido
print(f" Filas por mapa en '{first_match}' :")
for map_name in maps_in_match:
    count = len(first_match_data[first_match_data['Map'] == map_name])
    print(f" - {map_name}: {count} filas")

 Análisis de mapas

 Mapas únicos en el juego:  ['Haven' 'All Maps' 'Ascent' 'Abyss' 'Bind' 'Lotus' 'Sunset' 'Icebox'
 'Split' 'Breeze']
Mapas disponibles: ['Haven' 'All Maps' 'Ascent' 'Abyss' 'Bind' 'Lotus' 'Sunset' 'Icebox'
 'Split' 'Breeze']

 Partido: Gen.G vs Sentinels
Mapas jugados: 8
Cuales: ['Haven' 'All Maps' 'Ascent' 'Lotus' 'Icebox' 'Split' 'Breeze' 'Bind']
 Valores de Side: ['both' 'attack' 'defend']
 Filas por mapa en 'Gen.G vs Sentinels' :
 - Haven: 30 filas
 - All Maps: 120 filas
 - Ascent: 60 filas
 - Lotus: 60 filas
 - Icebox: 60 filas
 - Split: 60 filas
 - Breeze: 60 filas
 - Bind: 30 filas


In [ ]:
def load_year_data(yaer):
    base_path = f'../data/raw/datos/vct_{year}/'

    print (f" Cargando {year}...")

    try:
        # Cargamos los archivos
        overview = pd.read_csv(f'{base_path}matches/overview.csv')
        scores = pd.read_csv(f'{base_path}matches/scores.csv')
        maps_scores = pd.read_csv(f'{base_path}matches/maps_scores.csv')

        # Añadimos la columna de año
        overview['year'] = year
        scores['year'] = year
        maps_scores['year'] = year

        print(f"Overview: {len(overview):,} filas")
        print(f"Scores: {len(scores):,} filas")
        print(f"Maps Scores: {len(maps_scores):,} filas")

        return{
            'overview': overview,
            'scores': scores,
            'maps_scores': maps_scores
        }
    except FileNotFoundError as e:
        print("Error: Archivo no encontrado")
        return None
    except Exception as e:
        print(f"Error:{e}")
        return None


print("=" *60)
print(" Cargando todos los años...")
print("=" *60)

years = [2021, 2022, 2023, 2024, 2025]

all_data = {}

for year in years:
    data = load_year_data(year)
    if data is not None:
        all_data[year] = data
    print()

print("=" * 60)
print(f"Carga completa: {len(all_data)} años cargados")
print("=" * 60)

for year in all_data.keys():
    print(f"{year}")

 Cargando todos los años...
 Cargando 2021...
Overview: 599,355 filas
Scores: 7,224 filas
Maps Scores: 14,503 filas

 Cargando 2022...
Overview: 379,378 filas
Scores: 3,842 filas
Maps Scores: 8,884 filas

 Cargando 2023...
Overview: 34,944 filas
Scores: 331 filas
Maps Scores: 830 filas

 Cargando 2024...
Overview: 46,152 filas
Scores: 434 filas
Maps Scores: 1,104 filas

 Cargando 2025...
Overview: 53,226 filas
Scores: 503 filas
Maps Scores: 1,277 filas

Carga completa: 5 años cargados
2021
2022
2023
2024
2025


In [ ]:
print("Juntar el overview de todos los años")
print("="*60)

overviwe_list = []

for year in all_data.keys():
    overviwe_list.append(all_data[year]['overview'])

# Juntamos todos los overview en un solo dataframe
df_overview_all = pd.concat(overviwe_list, ignore_index = True)

print(f"Overview consolidado creado")
print(f"\n Resumen")
print(f"- Total de filas: {len(df_overview_all):,}")
print(f"- Columnas: {df_overview_all.shape[1]}")
print(f" - Años: {sorted(df_overview_all['year'].unique())}")

# Vemos como se distribuyen por año
year_counts = df_overview_all['year'].value_counts().sort_index()
for year, count in year_counts.items():
    print(f"- {year}: {count:,} filas")


print(f"\n Primeras 3 filas:")
display(df_overview_all[['year', 'Tournament', 'Match Name', 'Player', 'Team', 'Kills', 'Deaths']].head(3))

Juntar el overview de todos los años
Overview consolidado creado

 Resumen
- Total de filas: 1,113,055
- Columnas: 22
 - Años: [2021, 2022, 2023, 2024, 2025]
- 2021: 599,355 filas
- 2022: 379,378 filas
- 2023: 34,944 filas
- 2024: 46,152 filas
- 2025: 53,226 filas

 Primeras 3 filas:


,year,Tournament,Match Name,Player,Team,Kills,Deaths
0,2021,Valorant Champions 2021,Vision Strikers vs FULL SENSE,MaKo,Vision Strikers,13.0,8.0
1,2021,Valorant Champions 2021,Vision Strikers vs FULL SENSE,MaKo,Vision Strikers,9.0,5.0
2,2021,Valorant Champions 2021,Vision Strikers vs FULL SENSE,MaKo,Vision Strikers,4.0,3.0


In [ ]:
# Juntamos scores y maps_scores
print(" Juntamos scores y maps_scores")
print("="*60)
print("\n Scores:")

scores_list = []
for year in all_data.keys():
    scores_list.append(all_data[year]['scores'])

df_scores_all = pd.concat(scores_list, ignore_index = True)

print(f"Total: {len(df_scores_all):,} filas")
print(f"Años: {sorted(df_scores_all['year'].unique())}")

print(f"\n Primeras 3 filas:")
display(df_scores_all[['year', 'Match Name', 'Team A', 'Team B', 'Team A Score', 'Team B Score', 'Match Result']].head(3))


print("\n Maps scores:")
maps_scores_list = []
for year in all_data.keys():
    maps_scores_list.append(all_data[year]['maps_scores'])

df_maps_scores_all = pd.concat(maps_scores_list, ignore_index = True)

print(f"Total: {len(df_maps_scores_all):,} filas")
print(f"Años: {sorted(df_maps_scores_all['year'].unique())}")




print(f"\nPrimeras 3 filas:")
display(df_maps_scores_all[['year', 'Match Name', 'Map', 'Team A', 'Team B', 'Team A Score', 'Team B Score']].head(3))

print("\n" + "=" * 60)
print("Consolidación competada")
print("=" * 60)

 Juntamos scores y maps_scores

 Scores:
Total: 12,334 filas
Años: [2021, 2022, 2023, 2024, 2025]

 Primeras 3 filas:


,year,Match Name,Team A,Team B,Team A Score,Team B Score,Match Result
0,2021,Vision Strikers vs FULL SENSE,Vision Strikers,FULL SENSE,2,0,Vision Strikers won
1,2021,Team Vikings vs Crazy Raccoon,Team Vikings,Crazy Raccoon,2,0,Team Vikings won
2,2021,FNATIC vs Cloud9,FNATIC,Cloud9,2,1,FNATIC won



 Maps scores:
Total: 26,598 filas
Años: [2021, 2022, 2023, 2024, 2025]

Primeras 3 filas:


,year,Match Name,Map,Team A,Team B,Team A Score,Team B Score
0,2021,Vision Strikers vs FULL SENSE,Haven,Vision Strikers,FULL SENSE,13,5
1,2021,Vision Strikers vs FULL SENSE,Breeze,Vision Strikers,FULL SENSE,13,5
2,2021,Team Vikings vs Crazy Raccoon,Icebox,Team Vikings,Crazy Raccoon,13,9



Consolidación competada


In [ ]:

print(" Guardando datos consolidados")
print("="*60)

output_path = "../data/processed/"
os.makedirs(output_path, exist_ok = True)
print("\n Guardando datos....")


df_overview_all.to_parquet(f'{output_path}overview_all_years.parquet', index=False)
print(f"overview_all_years.parquet guardado")


df_scores_all.to_parquet(f'{output_path}scores_all_years.parquet', index=False)
print(f"scores_all_years.parquet guardado")


df_maps_scores_all.to_parquet(f'{output_path}maps_scores_all_years.parquet', index=False)
print(f"maps_scores_all_years.parquet guardado")

print("\n" + "=" * 60)
print("Archivos guardados en: ../data/processed/")
print("=" * 60)

 Guardando datos consolidados

 Guardando datos....
overview_all_years.parquet guardado
scores_all_years.parquet guardado
maps_scores_all_years.parquet guardado

Archivos guardados en: ../data/processed/
